___

**Execute these each time before program execution**

**Google colab doesn't save files so we have to upload audio and csv file each time we logged in.** 

___

pip install speaker-verification-toolkit                                
pip install numba==0.48                                                   
pip install resampy==0.3.1


In [ ]:
from tkinter.ttk import Style
import speaker_verification_toolkit as svt
import librosa
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from glob import glob
import librosa.display
import IPython.display as ipd
from itertools import cycle
import seaborn as sns
sns.set_theme(style="white",palette=None)
color_pal=plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle=cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])


In [ ]:
audio_files=glob('../content/Audio.wav')
ipd.Audio(audio_files[0])

In [ ]:
y,h=librosa.load(audio_files[0])
pd.Series(y).plot(figsize=(10,5),title="Raw Audio")
plt.show()

In [ ]:
yt, _=librosa.effects.trim(y,top_db=20)
pd.Series(yt).plot(figsize=(10,5),title="Edited Audio",color=color_pal[1])
plt.show()

In [ ]:
D=librosa.stft(y)
S_db=librosa.amplitude_to_db(np.abs(D),ref=np.max)
fig,ax=plt.subplots(figsize=(10,5))
img=librosa.display.specshow(S_db,x_axis='time',y_axis='log',ax=ax)
ax.set_title('Spectogram',fontsize=20)
plt.show()

In [ ]:
S=librosa.feature.melspectrogram(y=y,sr=h,n_mels=120,)
S_db_mel =librosa.amplitude_to_db(np.abs(S),ref=np.max)
fig,ax=plt.subplots(figsize=(15,5))
img=librosa.display.specshow(S_db_mel,x_axis='time',y_axis='log',ax=ax)
ax.set_title('Mel Spectogram of cough sound',fontsize=20)
plt.show()

In [ ]:
import librosa
import speaker_verification_toolkit.tools as svt
data,sr = librosa.load('Audio.wav',sr=16000,mono='true')

data = svt.rms_silence_filter(data)

data = svt.extract_mfcc(data)

print("MFCC Values:\n",data[1])

In [ ]:
FRAME_SIZE = 1024
HOP_LENGTH = 512
sc_debussy = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=FRAME_SIZE, hop_length=HOP_LENGTH)[0]
ban_debussy = librosa.feature.spectral_bandwidth(y=y, sr=sr, n_fft=FRAME_SIZE, hop_length=HOP_LENGTH)[0]
rolloff_debussy=librosa.feature.spectral_rolloff(y=y,sr=sr,n_fft=FRAME_SIZE, hop_length=HOP_LENGTH)[0]
frames = range(len(sc_debussy))
t = librosa.frames_to_time(frames, hop_length=HOP_LENGTH)
len(t)
plt.figure(figsize=(25,10))
plt.plot(t, rolloff_debussy,'y')
plt.plot(t, sc_debussy, color='b')
plt.plot(t, ban_debussy,'--r')
plt.legend(["Spectral RollOff","Spectral centroid","Spectral Bandwidth"],loc="lower right",fontsize="x-large")
plt.show()


print("The values of spectral centroid, spectral bandwidth, spectral rolloff are given below:")
speccen=sum(ban_debussy)/len(ban_debussy)
specban=sum(sc_debussy)/len(sc_debussy)
specroll=sum(rolloff_debussy)/len(rolloff_debussy)
print("Spectral Bandwidth :",speccen)
print("Spectral Centroid  :",specban)
print("Spectral RollOff   :",specroll)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

df = pd.read_csv('../content/FEFile.csv')
df.head()
df = df[['spectral_centroid', 'spectral_bandwidth', 'rolloff', 'mfcc1', 'mfcc2',
'mfcc3', 'mfcc5', 'mfcc6', 'mfcc8', 'mfcc12', 'mfcc14', 'mfcc21',
'mfcc30', 'mfcc32', 'mfcc34', 'mfcc36', 'label']]
df.head()
x = df.drop(['label'], axis=1)
y = df['label']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=100)

In [ ]:
dts = DecisionTreeClassifier()
dts.fit(x_train, y_train)
y_pred = dts.predict(x_test)
ac_dts = accuracy_score(y_pred, y_test)
ac_dts = ac_dts * 100
msg = "The accuracy obtained by Decision tree Clasiifier " + str(ac_dts) + str('%')
print(msg)
lee=list(range(3))
lee[0],lee[1],lee[2]=float(speccen),float(specban),float(specroll)
floats = [float(np_float) for np_float in data[1]]
lee=lee+floats


In [ ]:
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
result = model.predict([lee])
print(result)
result = result[0]
if result == 'covid':
    msg = 'The Person Has the Covid Disease, Please Consult With A Doctor 😲'
    print(msg)
else:
    msg = 'You Don\'t Have Covid, Enjoy Your Day 😜'
    print(msg)